# Use the results from magnet training to compute TPR, FPR and AUC

In [1]:
import numpy as np
from scipy.stats import entropy
from scipy import ndimage

In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torch.utils.data import DataLoader
import torchvision.transforms as transforms

In [3]:
def train_fs(distances, train_fpr):
    selected_distance_idx = int(np.ceil(len(distances) * (1-train_fpr)))
    threshold = sorted(distances)[selected_distance_idx-1]
    threshold = threshold
    #print ("Threshold value: %f" % threshold)
    return threshold

In [4]:
def test(model, dataset, X, threshold):
    distances = get_distance_test(model, dataset, X)
    Y_pred = distances > threshold
    return Y_pred, distances

In [5]:
def compute_distance(model, dataset, X):
    distances = get_distance_test(model, dataset, X)
    return distances

In [6]:
def get_tpr_fpr(true_labels, pred_labels):
    TP = np.sum(np.logical_and(pred_labels == 1, true_labels == 1))
    FP = np.sum(np.logical_and(pred_labels == 1, true_labels == 0))

    AP = np.sum(true_labels)
    AN = np.sum(1-true_labels)

    tpr = TP/AP if AP>0 else np.nan
    fpr = FP/AN if AN>0 else np.nan

    return tpr, fpr, TP, AP, FP, AN

In [7]:
def evaluate_test(model, x_test, threshold, dataset):
    Y_all = np.concatenate([np.ones(len(x_test), dtype=bool)])
    #print(Y_all)
    Y_all_pred, Y_all_pred_score = test(model, dataset, x_test, threshold)
    tpr, fpr, tp, ap, fp, an = get_tpr_fpr(Y_all, Y_all_pred)
    return tpr
                                           

# CIFAR thresholds

In [8]:
import pandas as pd
import numpy as np

In [16]:
#train data
df_cifar = pd.read_csv("cifar/Benign_CIFAR_Metrics.csv")

In [17]:
recon_error_benign = df_cifar.iloc[16].values.flatten().tolist()[1:]

In [18]:
min(recon_error_benign), max(recon_error_benign)

(0.0115477889776229, 0.1561983227729797)

In [19]:
fpr = [0.01, 0.05, 0.1] 
for train_fpr in fpr: 
    print(train_fs(recon_error_benign, train_fpr))

0.0777007564902305
0.0575042925775051
0.0491391010582447


In [20]:
threshold = [0.0777007564902305, 0.0575042925775051, 0.0491391010582447]

# now compute TPR

In [21]:
def compute_tpr(adv, threshold): 
    TN=0
    FN=0
    FP = 0 
    TP=0
    
    for value in adv: 
        if value>threshold:
            TP += 1
        else: 
            FN += 1
    
    
    return (TP/(TP+FN))*100

In [22]:
#pgd with five epsilons
df_pgd_eps1 = pd.read_csv('cifar/pgd/eps8_255.csv')
df_pgd_eps2 = pd.read_csv('cifar/pgd/eps16_255.csv')
df_pgd_eps3 = pd.read_csv('cifar/pgd/eps32_255.csv')
df_pgd_eps4 = pd.read_csv('cifar/pgd/eps64_255.csv')


#fgsm with five epsilons
df_fgsm_eps1 = pd.read_csv('cifar/fgsm/eps8_255.csv')
df_fgsm_eps2 = pd.read_csv('cifar/fgsm/eps16_255.csv')
df_fgsm_eps3 = pd.read_csv('cifar/fgsm/eps32_255.csv')
df_fgsm_eps4 = pd.read_csv('cifar/fgsm/eps64_255.csv')

#bim with five epsilons
df_bim_eps1 = pd.read_csv('cifar/bim/eps8_255.csv')
df_bim_eps2 = pd.read_csv('cifar/bim/eps16_255.csv')
df_bim_eps3 = pd.read_csv('cifar/bim/eps32_255.csv')
df_bim_eps4 = pd.read_csv('cifar/bim/eps64_255.csv')

#cw with four confidences
df_cw_conf1 = pd.read_csv('cifar/cw/conf_0.csv')
df_cw_conf2 = pd.read_csv('cifar/cw/conf_50.csv')

In [23]:
magnet1 = df_pgd_eps1.iloc[16].values.flatten().tolist()[1:]
magnet2 = df_pgd_eps2.iloc[16].values.flatten().tolist()[1:]
magnet3 = df_pgd_eps3.iloc[16].values.flatten().tolist()[1:]
magnet4 = df_pgd_eps4.iloc[16].values.flatten().tolist()[1:]

In [24]:
for th in threshold:
    TPR = compute_tpr(magnet1, th)
    print(TPR)
    TPR = compute_tpr(magnet2, th)
    print(TPR)
    TPR = compute_tpr(magnet3, th)
    print(TPR)
    TPR = compute_tpr(magnet4, th)
    print(TPR)
    print('---')

0.5988023952095809
1.1928429423459244
1.5810276679841897
100.0
---
3.792415169660679
3.9761431411530817
26.877470355731226
100.0
---
7.18562874251497
9.542743538767395
95.45454545454545
100.0
---


In [25]:
magnet1 = df_fgsm_eps1.iloc[16].values.flatten().tolist()[1:]
magnet2 = df_fgsm_eps2.iloc[16].values.flatten().tolist()[1:]
magnet3 = df_fgsm_eps3.iloc[16].values.flatten().tolist()[1:]
magnet4 = df_fgsm_eps4.iloc[16].values.flatten().tolist()[1:]


for th in threshold:
    TPR = compute_tpr(magnet1, th)
    print(TPR)
    TPR = compute_tpr(magnet2, th)
    print(TPR)
    TPR = compute_tpr(magnet3, th)
    print(TPR)
    TPR = compute_tpr(magnet4, th)
    print(TPR)
    print('---')

0.19685039370078738
2.549019607843137
27.218934911242602
100.0
---
4.133858267716536
10.980392156862745
98.42209072978304
100.0
---
12.795275590551181
24.313725490196077
100.0
100.0
---


In [26]:
magnet1 = df_bim_eps1.iloc[16].values.flatten().tolist()[1:]
magnet2 = df_bim_eps2.iloc[16].values.flatten().tolist()[1:]
magnet3 = df_bim_eps3.iloc[16].values.flatten().tolist()[1:]
magnet4 = df_fgsm_eps4.iloc[16].values.flatten().tolist()[1:]


for th in threshold:
    TPR = compute_tpr(magnet1, th)
    print(TPR)
    TPR = compute_tpr(magnet2, th)
    print(TPR)
    TPR = compute_tpr(magnet3, th)
    print(TPR)
    TPR = compute_tpr(magnet4, th)
    print(TPR)
    print('---')

0.9940357852882704
0.5964214711729622
2.549019607843137
100.0
---
4.7713717693836974
4.3737574552683895
66.66666666666666
100.0
---
11.332007952286283
12.922465208747516
100.0
100.0
---


In [27]:
magnet1 = df_cw_conf1.iloc[16].values.flatten().tolist()[1:]
magnet2 = df_cw_conf2.iloc[16].values.flatten().tolist()[1:]

for th in threshold:
    TPR = compute_tpr(magnet1, th)
    print(TPR)
    TPR = compute_tpr(magnet2, th)
    print(TPR)
    print('----')

1.3779527559055118
1.7786561264822136
----
4.724409448818897
5.138339920948617
----
8.070866141732283
11.067193675889328
----


# Check test FPR

In [29]:
df_cifar = pd.read_csv("cifar/Test_Data_Benign_CIFAR_Metrics.csv")

In [30]:
recon = df_cifar.iloc[16].values.flatten().tolist()[1:]
threshold = [0.0777007564902305, 0.0575042925775051, 0.0491391010582447]

In [31]:
def compute_FPR(ben, threshold): 
    TN=0
    FN=0
    FP = 0 
    TP=0
    
    for value in ben: 
        if value<threshold:
            TN += 1
        else: 
            FP += 1
    
    
    return (FP/(TN+FP))*100

In [32]:
for th in threshold:
    FPR = compute_FPR(recon, th)
    print(FPR)

0.8565737051792829
4.482071713147411
9.103585657370518


# Compute ROC AUC score

In [33]:
df_cifar = pd.read_csv("cifar/Test_Data_Benign_CIFAR_Metrics.csv")
recon = df_cifar.iloc[16].values.flatten().tolist()[1:]

In [34]:
min(recon), max(recon)

(0.0129460850730538, 0.1331789791584015)

In [35]:
threshold = [0.0777007564902305, 0.0575042925775051, 0.0491391010582447,0.048, 0.0407,0.0401,0.035,0.033,0.03,0.025,0.022,0.02,0.01]

In [36]:
fpr_results = []
for th in threshold:
    FPR = compute_FPR(recon, th)
    fpr_results.append(FPR/100)
print(fpr_results)

[0.008565737051792829, 0.044820717131474105, 0.09103585657370518, 0.10179282868525895, 0.1760956175298805, 0.18466135458167332, 0.27629482071713146, 0.3247011952191235, 0.41872509960159365, 0.6250996015936255, 0.7754980079681275, 0.8846613545816733, 1.0]


In [37]:
#pgd with five epsilons
df_pgd_eps1 = pd.read_csv('cifar/pgd/eps8_255.csv')
df_pgd_eps2 = pd.read_csv('cifar/pgd/eps16_255.csv')
df_pgd_eps3 = pd.read_csv('cifar/pgd/eps32_255.csv')
df_pgd_eps4 = pd.read_csv('cifar/pgd/eps64_255.csv')


#fgsm with five epsilons
df_fgsm_eps1 = pd.read_csv('cifar/fgsm/eps8_255.csv')
df_fgsm_eps2 = pd.read_csv('cifar/fgsm/eps16_255.csv')
df_fgsm_eps3 = pd.read_csv('cifar/fgsm/eps32_255.csv')
df_fgsm_eps4 = pd.read_csv('cifar/fgsm/eps64_255.csv')

#bim with five epsilons
df_bim_eps1 = pd.read_csv('cifar/bim/eps8_255.csv')
df_bim_eps2 = pd.read_csv('cifar/bim/eps16_255.csv')
df_bim_eps3 = pd.read_csv('cifar/bim/eps32_255.csv')
df_bim_eps4 = pd.read_csv('cifar/bim/eps64_255.csv')

#cw with four confidences
df_cw_conf1 = pd.read_csv('cifar/cw/conf_0.csv')
df_cw_conf2 = pd.read_csv('cifar/cw/conf_50.csv')

In [38]:
def compute_TPR(adv, threshold): 
    TN=0
    FN=0
    FP = 0 
    TP=0
    
    for value in adv: 
        if value>threshold:
            TP += 1
        else: 
            FN += 1
    
    
    return (TP/(TP+FN))*100

In [39]:
def compute_auc(adv_list, th): 
    tpr_results = []
    for th in threshold:
        TPR = compute_TPR(adv_list, th)
        tpr_results.append(TPR/100)
    return sklearn.metrics.auc(fpr_results, tpr_results)

In [40]:
import sklearn
from sklearn.metrics import roc_auc_score

feature_squeezer_1 = df_pgd_eps1.iloc[16].values.flatten().tolist()[1:]
feature_squeezer_2 = df_pgd_eps2.iloc[16].values.flatten().tolist()[1:]
feature_squeezer_3 = df_pgd_eps3.iloc[16].values.flatten().tolist()[1:]
feature_squeezer_4 = df_pgd_eps4.iloc[16].values.flatten().tolist()[1:]

print(compute_auc(feature_squeezer_1, threshold))
print(compute_auc(feature_squeezer_2, threshold))
print(compute_auc(feature_squeezer_3, threshold))
print(compute_auc(feature_squeezer_4, threshold))

0.512443638619176
0.7074530902235986
0.9418102294379793
0.9914342629482072


In [41]:
feature_squeezer_1 = df_fgsm_eps1.iloc[16].values.flatten().tolist()[1:]
feature_squeezer_2 = df_fgsm_eps2.iloc[16].values.flatten().tolist()[1:]
feature_squeezer_3 = df_fgsm_eps3.iloc[16].values.flatten().tolist()[1:]
feature_squeezer_4 = df_fgsm_eps4.iloc[16].values.flatten().tolist()[1:]

print(compute_auc(feature_squeezer_1, threshold))
print(compute_auc(feature_squeezer_2, threshold))
print(compute_auc(feature_squeezer_3, threshold))
print(compute_auc(feature_squeezer_4, threshold))

0.6488049769426232
0.838624521521756
0.9775902307928052
0.9914342629482072


In [42]:
feature_squeezer_1 = df_bim_eps1.iloc[16].values.flatten().tolist()[1:]
feature_squeezer_2 = df_bim_eps2.iloc[16].values.flatten().tolist()[1:]
feature_squeezer_3 = df_bim_eps3.iloc[16].values.flatten().tolist()[1:]
feature_squeezer_4 = df_bim_eps4.iloc[16].values.flatten().tolist()[1:]

print(compute_auc(feature_squeezer_1, threshold))
print(compute_auc(feature_squeezer_2, threshold))
print(compute_auc(feature_squeezer_3, threshold))
print(compute_auc(feature_squeezer_4, threshold))

0.6410707072307192
0.8148564786579329
0.9600238262635731
0.9913630350895045


In [43]:
feature_squeezer_1 = df_cw_conf1.iloc[16].values.flatten().tolist()[1:]
feature_squeezer_2 = df_cw_conf2.iloc[16].values.flatten().tolist()[1:]

print(compute_auc(feature_squeezer_1, threshold))
print(compute_auc(feature_squeezer_2, threshold))

0.49855989428114317
0.5219322315481159
